## Visualise stokes cartesian model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

[Lyrebird.local:09660] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.Lyrebird.501/jf.0/353173504/sm_segment.Lyrebird.501.150d0000.0 could be created.


In [3]:
ls -trl output/cartesian* 

output/cartesian:
total 0
drwxr-xr-x@  294 lmoresi  staff    9408 Aug 26 21:59 Ra1e3.0/
drwxr-xr-x@  909 lmoresi  staff   29088 Aug 27 16:34 Ra1e4.0/
drwxr-xr-x@ 2009 lmoresi  staff   64288 Aug 27 18:41 Ra1e5.0/
drwxr-xr-x  7517 lmoresi  staff  240544 Aug 28 21:19 Ra1e7.0/
drwxr-xr-x  6758 lmoresi  staff  216256 Aug 29 11:44 Ra1e8.0/
drwxr-xr-x@ 6818 lmoresi  staff  218176 Aug 29 11:56 Ra1e6.0/

output/cartesian_2x1:
total 0
drwxr-xr-x  6259 lmoresi  staff  200288 Aug 31 10:40 Ra1e6.0/

output/cartesian_1x1:
total 0
drwxr-xr-x@  7022 lmoresi  staff  224704 Sep  6 19:25 Ra1e6.0/
drwxr-xr-x@ 18003 lmoresi  staff  576096 Sep  7 14:50 Ra1e8.0/
drwxr-xr-x@ 19121 lmoresi  staff  611872 Sep  7 18:16 Ra1e7.0/


In [4]:
! ls -trl output/cartesian_1x1/Ra1e7.0/ | tail

-rw-r--r--@    1 lmoresi  staff   61432 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.U.00626.h5
-rw-r--r--@    1 lmoresi  staff   28360 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.P.00626.h5
-rw-r--r--@    1 lmoresi  staff  101480 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.T.00626.h5
-rw-r--r--@    1 lmoresi  staff   28360 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.eta_n.00626.h5
-rw-r--r--@    1 lmoresi  staff    3801 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.00626.xdmf
-rw-r--r--@    1 lmoresi  staff   61432 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.U.00627.h5
-rw-r--r--@    1 lmoresi  staff   28360 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.P.00627.h5
-rw-r--r--@    1 lmoresi  staff  101480 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.T.00627.h5
-rw-r--r--@    1 lmoresi  staff   28360 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.eta_n.00627.h5
-rw-r--r--@    1 lmoresi  staff    3801 Sep  7 18:16 tau_y_ii_Ra1e7.0_visc4.5_res15.mesh.006

In [5]:
# This is a bit repetitive, sorry 

checkpoint_dir = "output/cartesian_1x1/Ra1e7.0/"
name = "tau_y_ii_Ra1e7.0_visc4.5"
resolution = 15

checkpoint_base = f"{name}_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"_images", name)
os.makedirs(image_dir, exist_ok=True)

In [6]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
eta_soln = uw.discretisation.MeshVariable("\eta_n", mesh, 1, degree=1)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1, continuous=False)

In [7]:
flux_solver = uw.systems.Projection(mesh, flux)

flux_fn = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_1) + v_soln.sym[1] * t_soln.sym[0]
flux_solver.uw_function = flux_fn
flux_solver.smoothing = 0.0


sample_points = np.zeros((100,2))
sample_points[:,0] = np.linspace(0,1,100)
sample_points[:,1] = 1

sample_points_surf = sample_points.copy()

sample_points[:,1] = 0
sample_points_base = sample_points.copy()



In [8]:
flux_solver.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Projection'>

**Poisson system solver**

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

This solver is formulated in 2 dimensions

In [67]:
steps = range(700,800, 1)
calc_flux = True
add_streamlines = True
cmap = "RdBu_r"

In [68]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[750, 750])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        eta_soln.read_timestep(checkpoint_base, "eta_n", step, outputPath=checkpoint_dir)
        if not calc_flux:
            print(f"Plotting {step:05d}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
    pvmesh.point_data["Eta"] = vis.scalar_fn_to_pv_points(pvmesh, eta_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym[0])
    
    # point sources at cell centres
    skip = 19
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)


    pl.add_mesh(
        pvmesh_t,
        cmap=cmap,
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=False,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        clim=[0.0,1.0],
        show_scalar_bar=False
    )


    pl.add_mesh(
        pvmesh,
        cmap="Greys",
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=False,
        scalars="Eta",
        use_transparency=False,
        opacity= [0, 0.0, 0.2,  0.5],
        clim=[1.0,30000.0],
        show_scalar_bar=False
    )


    # pl.add_mesh(pvmesh, cmap="RdBu_r", scalars="T", opacity=0.5, show_edges=True, edge_color="Black",
    #     edge_opacity=0.5, show_scalar_bar=True)   

    if add_streamlines:
        pvstream = pvmesh.streamlines_from_source(
                point_cloud,
                vectors="V",
                integration_direction="both",
                max_time=1,
                surface_streamlines=True,
            )
    
        pl.add_mesh(pvstream,
                    opacity=0.5,
                    show_scalar_bar=False)

    # pl.camera_position = [(3.0, 0.5, 6.750994846396983),
    #                       (3.0, 0.5, 0.0),
    #                       (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 1000),
                  return_img=False)

    if calc_flux:
        flux_solver.solve()
        surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
        basal_flux = uw.function.evaluate(flux.sym, sample_points_base)
    
        # Heat flux: Upper, Lower, Mean
        print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}")




00700,    6.9124,   8.0947,   7.5036
00701,    6.9878,   8.0930,   7.5404
00702,    7.0637,   8.0906,   7.5772
00703,    7.1401,   8.0879,   7.6140
00704,    7.2168,   8.0844,   7.6506
00705,    7.2940,   8.0802,   7.6871
00706,    7.3714,   8.0757,   7.7235
00707,    7.4494,   8.0702,   7.7598
00708,    7.5287,   8.0637,   7.7962
00709,    7.6096,   8.0559,   7.8327
00710,    7.6923,   8.0475,   7.8699
00711,    7.7775,   8.0388,   7.9082
00712,    7.8654,   8.0298,   7.9476
00713,    7.9565,   8.0205,   7.9885
00714,    8.0516,   8.0109,   8.0312
00715,    8.1510,   8.0011,   8.0760
00716,    8.2552,   7.9913,   8.1233
00717,    8.3648,   7.9816,   8.1732
00718,    8.4804,   7.9720,   8.2262
00719,    8.6022,   7.9624,   8.2823
00720,    8.7305,   7.9528,   8.3416
00721,    8.8649,   7.9435,   8.4042
00722,    9.0053,   7.9349,   8.4701
00723,    9.1530,   7.9269,   8.5399
00724,    9.3106,   7.9188,   8.6147
00725,    9.4848,   7.9101,   8.6974
00726,    9.6815,   7.9002,   8.7908
0

In [66]:
pl.show(jupyter_backend="trame")

Widget(value='<iframe src="http://localhost:50906/index.html?ui=P_0x34bb2b050_25&reconnect=auto" class="pyvist…

In [ ]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [ ]:
t_soln.stats()

In [ ]:
I = uw.maths.Integral(mesh, t_soln.sym[0])
mean = I.evaluate()
mean

In [ ]:
I.fn = (t_soln.sym[0]-mean)**2
np.sqrt(I.evaluate())
l2 = np.sqrt(I.evaluate())
l2

In [ ]:
! open output/cartesian_1x1/Ra1e6.0/images

In [ ]:
0/0

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
flux_fn = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_1)  + v_soln.sym[1] * t_soln.sym[0]
flux_solver.uw_function = flux_fn
flux_solver.smoothing = 0.0
flux_solver.solve()

surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
basal_flux = uw.function.evaluate(flux.sym, sample_points_base)

# Heat flux: Upper, Lower, Mean
print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}")



In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=1.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="RdYlBu_r",
    edge_color="Black",
    show_edges=False,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

# imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

# pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR

pl.show()
